In [2]:
import numpy as np
import math
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import sys
import os

module_path = os.path.abspath(os.path.join('..', 'libairfoil'))
if module_path not in sys.path:
    sys.path.append(module_path)

# The libairfoil.parsec module is from the repository by mbodmer on GitHub
# GitHub repository: https://github.com/mbodmer/libairfoil/tree/master
import libairfoil.parsec

def load_coordinates(filename):
    """
    Load airfoil coordinates from a text file.
    
    filename: Path to the file containing coordinates.
    """
    coords = []
    with open(filename, 'r') as file:
        for line in file:
            x, z = map(float, line.split())
            coords.append((x, z))
    return coords

def objective_function(params, target_coords):
    """
    Objective function to minimize. It calculates the difference between the target coordinates and 
    the coordinates generated by the current set of PARSEC parameters.
    
    params: List of PARSEC parameters [r_le, X_up, Z_up, Z_XX_up, X_lo, Z_lo, Z_XX_lo, Z_te, dZ_te, alpha_te, beta_te]
    target_coords: List of target coordinates as tuples [(x1, z1), (x2, z2), ..., (xn, zn)]
    """
    parsec_params = libairfoil.parsec.Parameters()
    parsec_params.r_le = params[0]
    parsec_params.X_up = params[1]
    parsec_params.Z_up = params[2]
    parsec_params.Z_XX_up = params[3]
    parsec_params.X_lo = params[4]
    parsec_params.Z_lo = params[5]
    parsec_params.Z_XX_lo = params[6]
    parsec_params.Z_te = params[7]
    parsec_params.dZ_te = params[8]
    parsec_params.alpha_te = math.radians(params[9])
    parsec_params.beta_te = math.radians(params[10])

    airfoil = libairfoil.parsec.Airfoil(parsec_params)
    
    x_up = np.array([coord[0] for coord in target_coords[:87]])
    z_up_target = np.array([coord[1] for coord in target_coords[:87]])
    
    x_lo = np.array([coord[0] for coord in target_coords[87:]])
    z_lo_target = np.array([coord[1] for coord in target_coords[87:]])
    
    z_up_generated = airfoil.Z_up(x_up)
    z_lo_generated = airfoil.Z_lo(x_lo)
    
    diff_up = np.linalg.norm(z_up_generated - z_up_target)
    diff_lo = np.linalg.norm(z_lo_generated - z_lo_target)
    
    return diff_up + diff_lo

def fit_airfoil_to_coordinates(target_coords):
    """
    Fit an airfoil to the given target coordinates by optimizing the PARSEC parameters.
    
    target_coords: List of target coordinates as tuples [(x1, z1), (x2, z2), ..., (xn, zn)]
    """
    initial_params = [1.36781157e-02,3.29828126e-01,7.37049878e-02,-5.51671401e-01,2.35726325e-01,-3.60487282e-02,1.13115166e-01,1.14351216e-03,0.00000000e+00,0.00000000e+00,1.86259073e+01]
    
    bounds = [(0, 1), (0, 1), (0, 1), (-2, 2), (0, 1), (-2, 2), (-2, 2), (-1, 1), (0, 1), (-45, 45), (0, 45)]
    
    result = minimize(objective_function, initial_params, args=(target_coords,), bounds=bounds, method='L-BFGS-B')
    
    return result.x

if __name__ == '__main__':
    filename = './coord_folder/1.txt'
    target_coords = load_coordinates(filename)
    
    optimized_params = fit_airfoil_to_coordinates(target_coords)
    print("Optimized PARSEC parameters:", optimized_params)

Optimized PARSEC parameters: [ 1.36705541e-02  3.47410750e-01  9.84000000e-02 -5.67271945e-01
  1.20080000e-01 -2.88300000e-02  1.09430658e-01  0.00000000e+00
  0.00000000e+00 -1.45671120e+00  1.86115056e+01]
